We now show how we can change the assembly by just editing the JSON file. We will drop the characteristic_length class since its output are not connected to another component, and also re-order the components in the assembly. The idea is that in the first instance we want to show how to remove an unused component if the study we want to do does not leverage any responses from this component.

This is an example of how we can now manipulate the creation of an OpenMDAO assembly purely via the JSON file. For example, we could develop a GUI that creates the JSON file, or write a converter that takes the structure assembled in ModelCenter or even Cameo and then instantiate the OpenMDAO model from that. Demonstrating these ideas is not part of this contract.

In [1]:
import json_numpy
import copy
file_name = 'demo_group.json'
with open(file_name, "r") as infile:
    full_info = json_numpy.load(infile)

In [2]:
new_info = copy.deepcopy(full_info)
# Show which components make up the prep_geom group
print(new_info['components']['prep_geom']['component_order'])
# Delete the 'characteristic_lengths' component from the order list.
del(new_info['components']['prep_geom']['component_order'][-2])
# We also want to move the 'nacelles' and 'canard' components to the start of the assembly
new_info['components']['prep_geom']['component_order'] =['nacelles', 'canard', 'fuselage_prelim', 'wing_prelim', 'prelim', 'wing', 'tail', 'fuselage', 'total_wetted_area']

# We now also delete the 'characteristic_lengths' component completely from the assembly
del(new_info['components']['prep_geom']['components']['characteristic_lengths'])
# We also need to delete the promotions. Note that we have to do this both for the 'prep_geom' group and the overall model.
del(new_info['components']['prep_geom']['promotions']['characteristic_lengths'])
del(new_info['promotions']['prep_geom'][3])
del(new_info['promotions']['prep_geom'][4])
del(new_info['promotions']['prep_geom'][77:89])

# Delete the entry in the linkage for a variable from 'characteristic_lenghts'
del(new_info['components']['prep_geom']['linkage'][-1])
# Delete the input variables for 'characteristic_lenghts'. 
del(new_info['components']['prep_geom']['inputs'][3])
del(new_info['components']['prep_geom']['inputs'][3])
del(new_info['inputs'][3])
del(new_info['inputs'][3])
# Delete the outputs from 'characteristic_lenghts'.
del(new_info['outputs'][40])
del(new_info['outputs'][1:13])
del(new_info['components']['prep_geom']['outputs'][1:13])

['fuselage_prelim', 'wing_prelim', 'prelim', 'wing', 'tail', 'fuselage', 'nacelles', 'canard', 'characteristic_lengths', 'total_wetted_area']


In [3]:
new_file_name = "after.json"
with open(new_file_name, "w") as outfile:
        json_numpy.dump(new_info, outfile, indent=4)


In [4]:
from  standard_evaluator import load_assembly
# Now loading the structure and state from these files
new_problem = load_assembly(assembly_name=new_file_name, state_name='state.h5')


** This is a group, special handling needed for prep_geom in group 
Adding nacelles component
Building class nacelles
Importing class Nacelles from aviary.subsystems.geometry.flops_based.nacelle
Adding canard component
Building class canard
Importing class Canard from aviary.subsystems.geometry.flops_based.canard
Adding fuselage_prelim component
Building class fuselage_prelim
Importing class FuselagePrelim from aviary.subsystems.geometry.flops_based.fuselage
Adding wing_prelim component
Building class wing_prelim
Importing class WingPrelim from aviary.subsystems.geometry.flops_based.wing
Adding prelim component
Building class prelim
Importing class _Prelim from aviary.subsystems.geometry.flops_based.prep_geom
Adding wing component
Building class wing
Importing class _Wing from aviary.subsystems.geometry.flops_based.prep_geom
Adding tail component
Building class tail
Importing class _Tail from aviary.subsystems.geometry.flops_based.prep_geom
Adding fuselage component
Building class fuse

In [5]:
_ = new_problem.model.list_inputs(shape=True, units=True)

new_problem.run_model()

59 Input(s) in 'model'

varname                                              val           units     shape  prom_name                                      
---------------------------------------------------  ------------  --------  -----  -----------------------------------------------
prep_geom
  nacelles
    aircraft:nacelle:avg_diameter                    [12.]         ft        (1,)   aircraft:nacelle:avg_diameter                  
    aircraft:nacelle:avg_length                      [23.]         ft        (1,)   aircraft:nacelle:avg_length                    
    aircraft:nacelle:wetted_area_scaler              [0.]          unitless  (1,)   aircraft:nacelle:wetted_area_scaler            
  canard
    aircraft:canard:area                             [3.]          ft**2     (1,)   aircraft:canard:area                           
    aircraft:canard:thickness_to_chord               [0.3]         unitless  (1,)   aircraft:canard:thickness_to_chord             
    aircraft:canard:we

In [6]:
_ = new_problem.model.list_outputs(shape=True)

25 Explicit Output(s) in 'model'

varname                                      val              shape  prom_name                              
-------------------------------------------  ---------------  -----  ---------------------------------------
prep_geom
  nacelles
    aircraft:nacelle:total_wetted_area       [0.]             (1,)   aircraft:nacelle:total_wetted_area     
    aircraft:nacelle:wetted_area             [0.]             (1,)   aircraft:nacelle:wetted_area           
  canard
    aircraft:canard:wetted_area              [5.07864]        (1,)   aircraft:canard:wetted_area            
  fuselage_prelim
    aircraft:fuselage:avg_diameter           [15.]            (1,)   aircraft:fuselage:avg_diameter         
    aircraft:fuselage:planform_area          [3600.]          (1,)   aircraft:fuselage:planform_area        
  wing_prelim
    aircraft:wing:aspect_ratio               [60.68268015]    (1,)   aircraft:wing:aspect_ratio             
  prelim
    prep_geom:_Names:CR

In [7]:
import openmdao.api as om

om.n2(new_problem, 'after.n2.html', display_in_notebook=False,  )
